In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #'last', 'last_expr'

In [3]:
import sys
sys.path.insert(0, '/home/hushengchun/python_library/TransformerLens/')

import transformer_lens
from argparse import ArgumentParser
from functools import partial 
from pathlib import Path

import pandas as pd
import numpy as np
import torch
from transformer_lens import HookedTransformer
import matplotlib.pyplot as plt
from tqdm import tqdm

from eap.graph import Graph
from eap.attribute import attribute, _plot_attn, tokenize_plus
from eap.evaluate import evaluate_graph, evaluate_baseline

from dataset import EAPDataset
from metrics import get_metric
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer

In [4]:
import einops

In [4]:
cache_dir = '/home/xd/.cache/torch/transformers/'
model_name = 'lmsys/vicuna-33b-v1.3'
proxies = {'http': '192.168.53.250:10801'}
device = 0
# model0 = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir, proxies=proxies,
#     local_files_only=False, low_cpu_mem_usage=True, use_safetensors=False, load_in_4bit=True, bnb_4bit_quant_type='nf4')
model0 = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir, proxies=proxies,
    local_files_only=False, low_cpu_mem_usage=True, use_safetensors=False)

In PreTrainedModel.from_pretrained: resolved_archive_file, filename = /home/xd/.cache/torch/transformers/132115a70169b8c5fbf5bf7aef5f9d6823f3853d163474fba15145c3d7460bcc.7f36ca25942bd037786a5dcc90e9a9659aa140a20766b90f2cbe4d623e54e261 pytorch_model.bin


In huggingface_hub.file_download.cached_download: url = https://huggingface.co/lmsys/vicuna-33b-v1.3/resolve/main/pytorch_model-00001-of-00007.bin
in cached_download: link_path = /home/xd/.cache/torch/transformers/vicuna-33b-v1.3-pytorch_model-00001-of-00007.bin
In huggingface_hub.file_download.cached_download: url = https://huggingface.co/lmsys/vicuna-33b-v1.3/resolve/main/pytorch_model-00002-of-00007.bin
in cached_download: link_path = /home/xd/.cache/torch/transformers/vicuna-33b-v1.3-pytorch_model-00002-of-00007.bin
In huggingface_hub.file_download.cached_download: url = https://huggingface.co/lmsys/vicuna-33b-v1.3/resolve/main/pytorch_model-00003-of-00007.bin
in cached_download: link_path = /home/xd/.cache/torch/transformers/vicuna-33b-v1.3-pytorch_model-00003-of-00007.bin
In huggingface_hub.file_download.cached_download: url = https://huggingface.co/lmsys/vicuna-33b-v1.3/resolve/main/pytorch_model-00004-of-00007.bin
in cached_download: link_path = /home/xd/.cache/torch/transforme

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
model = HookedTransformer.from_pretrained("llama-30b-hf",center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    fold_value_biases=False,
    device='cuda',
    hf_model=model0,
    tokenizer=tokenizer,
    # hf_model_4bit=True,
    dtype="float16",
)

Loaded pretrained model llama-30b-hf into HookedTransformer


In [7]:
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True

In [8]:
inputs = "The capital of China is"
tokens = model.to_tokens(inputs, prepend_bos=True, padding_side='right')

In [9]:
logit, cache = model.run_with_cache(tokens)

In [49]:
for a, b in cache.items():
    print(b)

tensor([[[-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
         [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
         [-0.0036,  0.0071,  0.0203,  ...,  0.0094,  0.0066,  0.0092],
         [ 0.0382,  0.0012, -0.0046,  ...,  0.0034,  0.0001,  0.0020],
         [ 0.0014,  0.0011,  0.0153,  ..., -0.0240, -0.0133, -0.0125],
         [ 0.0081,  0.0036,  0.0435,  ..., -0.0115,  0.0019, -0.0101]]],
       device='cuda:0', dtype=torch.float16)
tensor([[[-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
         [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
         [-0.0036,  0.0071,  0.0203,  ...,  0.0094,  0.0066,  0.0092],
         [ 0.0382,  0.0012, -0.0046,  ...,  0.0034,  0.0001,  0.0020],
         [ 0.0014,  0.0011,  0.0153,  ..., -0.0240, -0.0133, -0.0125],
         [ 0.0081,  0.0036,  0.0435,  ..., -0.0115,  0.0019, -0.0101]]],
       device='cuda:0', dtype=torch.float16)
tensor([[[0.0118],
         [0.0129],
         [0.0166

In [10]:
cache['blocks.0.hook_k_input']

tensor([[[[-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          ...,
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162]],

         [[-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          ...,
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001]],

         [[-0.0036,  0.0071,  0.0203,  ...,  0.0094,  0.0066,  0.0092],
          [-0.0036,  0.0071,  

In [11]:
query_true = cache["blocks.0.hook_q_input"]
query_true

tensor([[[[-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          ...,
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162]],

         [[-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          ...,
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001]],

         [[-0.0036,  0.0071,  0.0203,  ...,  0.0094,  0.0066,  0.0092],
          [-0.0036,  0.0071,  

In [21]:
scale = cache[f'blocks.{0}.ln1.hook_scale']
norm = cache[f'blocks.{0}.ln1.hook_normalized']

In [13]:
query = norm * scale
query = einops.repeat(
        query,
        "batch pos d_model -> batch pos n_heads d_model",
        n_heads=model.cfg.n_heads,
    )
query

tensor([[[[-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          ...,
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162],
          [-0.0195, -0.0037, -0.0011,  ...,  0.0023, -0.0019, -0.0162]],

         [[-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          ...,
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001],
          [-0.0142, -0.0049,  0.0112,  ..., -0.0172,  0.0081,  0.0001]],

         [[-0.0036,  0.0071,  0.0203,  ...,  0.0094,  0.0066,  0.0092],
          [-0.0036,  0.0071,  

In [12]:
import pandas as pd 

from transformers import AutoTokenizer

from data.ioi.ioi_dataset import IOIDataset
from eap.utils import model2family

model_name = 'gpt2'
model_name_noslash = model_name.split('/')[-1]
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


/home/hushengchun/project/eap-ig-faithfulness/data/ioi/ioi_dataset.py:770: UserWarning: Some groups have less than 5 prompts, they have lengths [3]
  warnings.warn(


In [ ]:
ds = IOIDataset('mixed', N=100, tokenizer=tokenizer)

In [13]:
abc_dataset = (  # TODO seeded
    ds.gen_flipped_prompts(("S", "RAND"))
)

In [14]:
abc_dataset = (abc_dataset.gen_flipped_prompts(("IO", "RAND")))

/home/hushengchun/project/eap-ig-faithfulness/data/ioi/ioi_dataset.py:501: UserWarning: S2 index has been computed as the same for S and S2
  warnings.warn("S2 index has been computed as the same for S and S2")


In [15]:
abc_dataset = (abc_dataset.gen_flipped_prompts(("S2", "RAND")))

In [16]:
d = {'clean': [], 'corrupted': [], 'correct_idx': [], 'incorrect_idx': []}
for i in range(len(ds)):
    clean = ' '.join(ds.sentences[i].split()[:-1])
    corrupted = ' '.join(abc_dataset.sentences[i].split()[:-1])
    correct = ds.toks[i, ds.word_idx['IO'][i]].item()
    incorrect = abc_dataset.toks[i, abc_dataset.word_idx['IO'][i]].item()
    d['clean'].append(clean)
    d['corrupted'].append(corrupted)
    d['correct_idx'].append(correct)
    d['incorrect_idx'].append(incorrect)

df = pd.DataFrame.from_dict(d)
df = df.sample(frac=1)
df.to_csv(f'{model2family(model_name)}_1.csv')